In [1]:
import sys 
sys.path.insert(0, '../DataCleaning')

import train_test_split

In [13]:
import importlib
importlib.reload(train_test_split)

<module 'train_test_split' from '../DataCleaning\\train_test_split.py'>

In [2]:
DATA_DIR = '../../data'

In [4]:
import json

with open('../../data/production_data/site_metadata.json', 'r') as file:
    s = file.read()
    site_md = json.loads(s)

In [15]:
from os.path import exists

splits = {}

#Warning, this takes a while because the join operation is implemented inefficently (like n^2 in the worst case.)
for site in site_md:
    site_id = site['id']
    irradiance_site_id = site['irradiance_site_id']
    tz_str = site['location']['timeZone']
    
    path_production = f"{DATA_DIR}/production_data/{site_id}/combination_data/production_weather_combination.csv"
    path_irradiance = f"{DATA_DIR}/irradiance_data/{irradiance_site_id}/irradiance_data.csv"
    
    if not exists(path_irradiance):
        print(site_id,irradiance_site_id)
        continue
    
    splits[site_id] = train_test_split.get_irradiance_WPI_data(path_production,path_irradiance, 4,3,tz_str)
    
    

1232644 116439


In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def mean_absoluate_percentage_error(y_true,y_pred):
    
    return np.mean( np.abs((y_true-y_pred) / y_true) ) * 100

In [20]:
for site_id,data in splits.items():
    
    model = LinearRegression()
    
    ([X_train,X_test],[Y_train,Y_test]) = data
    
    model.fit(X_train,Y_train)
    
    Y_pred = model.predict(X_test)
    
    mse = mean_squared_error(Y_test,Y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(Y_test,Y_pred)
    mape = mean_absoluate_percentage_error(Y_test,Y_pred)
    r2 = r2_score(Y_test,Y_pred)
    
    print(f'System {site_id}:')

    print(f'\t Root Mean Squared Error: {rmse}')
    print()

    print(f'\t Mean Absolute Error: {mae}')
    print(f'\t Mean Absolute Percentage Error: {mape}')
    print()

    print(f'\t r2 score: {r2}')

System 103941:
	 Root Mean Squared Error: 241.26352315655876

	 Mean Absolute Error: 172.76517497597231
	 Mean Absolute Percentage Error: 1168.6176666931142

	 r2 score: 0.8767377668554277
System 787197:
	 Root Mean Squared Error: 462.018796169785

	 Mean Absolute Error: 337.44544264622704
	 Mean Absolute Percentage Error: 305.91356865907056

	 r2 score: 0.8191481578022572
System 238320:
	 Root Mean Squared Error: 169.91785231311124

	 Mean Absolute Error: 136.9776718147462
	 Mean Absolute Percentage Error: 8539.3467608007

	 r2 score: 0.8931897128995506
System 349060:
	 Root Mean Squared Error: 180.5489742491527

	 Mean Absolute Error: 119.97947739857499
	 Mean Absolute Percentage Error: 91.98863965541429

	 r2 score: 0.9220784753952466
System 477834:
	 Root Mean Squared Error: 157.69286659561988

	 Mean Absolute Error: 121.83352336000353
	 Mean Absolute Percentage Error: 287.7390383135082

	 r2 score: 0.7558964436353491
System 641826:
	 Root Mean Squared Error: 1142.7821070169734

	 